# Scikit-learn 3

Feature extraction for text

## 0 Feature extraction

![title](food.jpg)

If you worked through the notebooks 1 and 2, you know how to load data sets, how to fit a machine learning model and how to evaluate it automatically. We assumed that we have a matrix `X` which represents our training observations, and that those would be in a suitable format. Exactly the same applies for the values in `y`.

In reality, we might have some data, but not in a format that is suitable for machine learning. This notebook describes ways to go from the raw data to reasonable representations in `X` and `y`. This process is called **feature extraction** (well, for the `X` part at least).

## 1 Extracting features from text (beware: spooky!)

Naturally, we are interested in extracting useful features from _text_. As the data set we do not use one of the prepared data sets that come with `scikit-learn` of course, but from a recent Kaggle challenge. Go to the Kaggle page:

https://www.kaggle.com/c/spooky-author-identification

download the training, test data and the sample submission, and unzip the files. In the following, I assume you have placed those files in the same folder as the notebook. As usual, we read data into memory with pandas:

In [18]:
import pandas as pd
df = pd.read_csv('train.csv', index_col=0) # first column are indexes only
df.head()

,text,author
id,,
id26305,"This process, however, afforded me no means of...",EAP
id17569,It never once occurred to me that the fumbling...,HPL
id11008,"In his left hand was a gold snuff box, from wh...",EAP
id27763,How lovely is spring As we looked from Windsor...,MWS
id12958,"Finding nothing else, not even gold, the Super...",HPL


Also as usual, we check the shapes of objects and find the target classes:

In [19]:
# check shapes
print df.shape
# different response classes?
set(df.author)

(19579, 2)


{'EAP', 'HPL', 'MWS'}

As you can see, the only feature we currently have to describe samples is one string of text. That's not an optimal representation. For one thing, it should be possible for words (or characters) to contribute individually to predictions.

Second, strings are [_nominal_](https://en.wikipedia.org/wiki/Level_of_measurement) values: most classifiers require features to be numbers and, consequently, observations to be lists of numbers. Note: having a large number of features, i.e. describing each training example with a long list of numbers, is not a problem at all.

### Normalization

Before we even start converting strings to numbers, there are some sources of variation that we would like to eliminate right away. Most of those procedures can be described as **normalization**.

Please note: all methods described are not universally helpful. Depending on your task, they can also be detrimental because they remove important task-specific information. 

**Casing**

For instance, the fact that the first word in a sentence (say, "How") is capitalized means that "how" and "How" will be regarded as different words, which does not make sense. There are different variants of casing (e.g. lowercasing or truecasing), but most of the time, lowercasing makes sense.

Lowercasing is a predefined method of Python `str` objects, and is also easy for pandas frames:

In [20]:
df_lower = df.copy()
df_lower.text = df.text.str.lower()
df_lower.head()

,text,author
id,,
id26305,"this process, however, afforded me no means of...",EAP
id17569,it never once occurred to me that the fumbling...,HPL
id11008,"in his left hand was a gold snuff box, from wh...",EAP
id27763,how lovely is spring as we looked from windsor...,MWS
id12958,"finding nothing else, not even gold, the super...",HPL


In `scikit-learn`, lowercasing comes as an option for transformer classes like `sklearn.feature_extraction.text.CountVectorizer`, see below.

**Accents, diacritics, punctuation, whitespace**

Similarly, different spellings of the same word lead to entirely different entries in the vocabulary, if not taken care of. Accents or diacritics on characters are important here: people omit them simply because they are unfamiliar or cannot find them on the keyboard. Another frequent source of confusion are strange punctuation or whitespace symbols that have a separate Unicode representation.

Here is a [standard approach](https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-in-a-python-unicode-string) to remove all of them, which is converting them to ASCII characters:

In [23]:
import unidecode # if you do not have this module yet: conda install unidecode
s = u"Rasenmäher"
s_normalized = unidecode.unidecode(s)
s_normalized

'Rasenmaher'

As with casing, in `scikit-learn` there is rarely a need to do this by hand, as it is implemented in a number of classes, such as `sklearn.feature_extraction.text.CountVectorizer`, see below.

**Tokenization**

In raw strings of text, whitespace characters do not always indicate boundaries between tokens. **Tokenization simply means making sure that whitespace characters indicate token boundaries**. In a raw string like

    "Preemptive strikes and the war on Iraq: a critique of Bush administration"
    
splitting at whitespaces will result in `"Iraq:"` as one of the words. Here are two real-world examples of efficient tokenizers: the [Moses tokenizer](https://github.com/moses-smt/mosesdecoder/blob/master/scripts/tokenizer/tokenizer.perl) and [cutter tokenizer](https://gitlab.cl.uzh.ch/graen/cutter).

In `scikit-learn`, we usually have a different way to go about this. Rather than modifying the raw string to insert whitspace characters, we define what exactly should be regarded as a token. A token can be anything that can be described with a regular expression.

**Stopword removal**

Finally, some of the words might need to be removed alltogether. In a collection of texts, some words will occur in all of the documents, and their distribution in the documents will be about the same. Words like that are commonly called **stopwords**.

Words that occur in all examples are bad features, so we would not want to waste resources to compute and store them. In Python, you can simply compile a list of stopwords and remove them from your input strings or ignore them. `scikit-learn` lets you use a stopword list or define your own, see below.

### Extracting features that are order-agnostic

We now turn to actual feature extraction, which means turning normalized strings into lists of numbers. Language is inherently _sequential_, there is a natural order to the "items" in a text. Yet, ignoring the order (or, to put it another way, ignoring the fact that a string is a sequence) is simple and effective for many tasks.

Feature extraction methods that are **order-agnostic** essentially see a string as a _bag_ of items, for instance a bag of words or bag of characters.

**Token counts feature vectors**

A straightfoward method is to split a string into tokens, represent each example by a vector of the size of the token vocabulary, and in each vector dimension indicate how many times that token occurred in each example. This procedure is called **count vectorization**.

Here is a toy example with a very small input matrix:

In [2]:
# toy example
from sklearn.feature_extraction.text import CountVectorizer

X = ["This is a toy example", "This is another toy example ."]

count_vectorizer = CountVectorizer()
print count_vectorizer

X_vectorized = count_vectorizer.fit_transform(X)
X_vectorized

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


<2x5 sparse matrix of type '<type 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

The result is a sparse matrix that can be converted back to a usual array and the vectorizer vocabulary is an attribute:

In [4]:
print X_vectorized.toarray()
print count_vectorizer.vocabulary_.items()

[[0 1 1 1 1]
 [1 1 1 1 1]]
[(u'this', 3), (u'is', 2), (u'toy', 4), (u'example', 1), (u'another', 0)]


Now vectorizing our real spooky data:

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

# perhaps after normalizing the cells of df
X = df.text
y = df.author

count_vectorizer = CountVectorizer()
print count_vectorizer

X_vectorized = count_vectorizer.fit_transform(X)
X_vectorized

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


<19579x25068 sparse matrix of type '<type 'numpy.int64'>'
	with 429602 stored elements in Compressed Sparse Row format>

A `CountVectorizer` has many useful options for normalization, such as `lowercase`, `stop_words`, `strip_accents`, `tokenizer` and so on. Read the documentation to fine-tune the vectorizer behaviour. The result of the transformation is stored in a new format, a `scipy` compressed sparse row matrix. It is an efficient data structure because we expect most of the vector entries to be zero and hence, the matrix will be **sparse**.

**Question for you: Why will most entries in the vectors be zero?**

After fitting a transformer and transforming with it, both its attributes and the result can be inspected more closely:

In [32]:
# vocabulary (equals size of vectors)
print count_vectorizer.vocabulary_.items()[:10]

# convert to usual numpy array
X_vectorized.toarray()

[(u'fawn', 8429), (u'hordes', 10716), (u'foul', 9063), (u'tilton', 22327), (u'porphyrogene', 16733), (u'woods', 24791), (u'clotted', 3916), (u'spiders', 20679), (u'nordland', 14870), (u'hanging', 10169)]


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

As you can see, we did not change the overall structure of `X`, it is still a two-dimensional array: a list of training examples where each example is itself a list. But we managed to represent all strings as numbers in a meaningful way.

**More sophisticated TF/IDF token count feature vectors**

Simple count vectorization gives equal weight to all features (remember, features are tokens in the vocabulary in this case). But actually, it is not true that all token features are equally informative. You might want to use adaptive weighting like **term frequency / inverse document frequency (TF/IDF)** vectorization.

**Question for you: What is TF/IDF and how does it transform counts? If you cannot recall, please look it up.**

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
print tfidf_vectorizer

X_vectorized = tfidf_vectorizer.fit_transform(X)
X_vectorized.toarray()

TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

**Character-level instead of word-level features**

All methods we have looked at so far operate on the level of tokens. Exactly the same can be done on the level of individual characters, if there is reason to believe that changes within token are important features.

A toy example (notice how punctuation characters are not filtered out this time):

In [7]:
char_count_vectorizer = CountVectorizer(analyzer='char')
print char_count_vectorizer

X_toy = ["sample text", "more sample text !!!!!!!!"]
X_vectorized = char_count_vectorizer.fit_transform(X_toy)

print char_count_vectorizer.vocabulary_
X_vectorized.toarray()

CountVectorizer(analyzer='char', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
{u'a': 2, u' ': 0, u'e': 3, u'm': 5, u'l': 4, u'o': 6, u'p': 7, u's': 9, u'r': 8, u'!': 1, u't': 10, u'x': 11}


array([[1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 2, 1],
       [3, 8, 1, 3, 1, 2, 1, 1, 1, 1, 2, 1]])

And applied to our real data:

In [35]:
char_count_vectorizer = CountVectorizer(analyzer='char')
print char_count_vectorizer

X_vectorized = char_count_vectorizer.fit_transform(X)
print char_count_vectorizer.vocabulary_

CountVectorizer(analyzer='char', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
{u'j': 17, u'o': 22, u't': 27, u'n': 21, u' ': 0, u'"': 1, u"'": 2, u'p': 23, u'z': 33, u',': 3, u'.': 4, u'\u03b1': 50, u'\u03b4': 51, u'\u1f36': 57, u';': 6, u':': 5, u'\u03bd': 52, u'\u03bf': 53, u'?': 7, u'\u03c0': 54, u'\u03c3': 55, u'\u03c5': 56, u'f': 13, u'v': 29, u'\xfc': 49, u'g': 14, u'b': 9, u'e': 12, u'd': 11, u'a': 8, u'\xe0': 34, u'c': 10, u'\xe2': 35, u'\xe5': 37, u'\xe4': 36, u'\xe7': 39, u'\xe6': 38, u'\xe9': 41, u'\xe8': 40, u'\xeb': 43, u'\xea': 42, u'm': 20, u'l': 19, u'\xef': 45, u'\xee': 44, u'\xf1': 46, u'i': 16, u's': 26, u'r': 25, u'u': 28, u'\xf4': 47, u'w': 30, u'\xf6': 48, u'y': 32, u'x

**Question for you: This drastically reduces the size of the feature vectors. Why?**

### Extracting features that incorporate order

Strings are ordered sequences, and perhaps, features should keep information about ordering. A simple way of introducing this into vector counts is by counting **n-grams** instead of single tokens or characters. Such a model could be accurately described as a **bag of n-grams** model. There will be information about order encoded in the features, but it will be limited to the size of n-grams chosen by the user.

In [37]:
ngram_count_vectorizer = CountVectorizer(ngram_range=(1, 3))
print ngram_count_vectorizer

X_vectorized = ngram_count_vectorizer.fit_transform(X)
print ngram_count_vectorizer.vocabulary_.items()[:10]

CountVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern=u'(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
[(u'in phenomena beyond', 255420), (u'he entered with', 220706), (u'silken hose', 457580), (u'yield that', 638692), (u'contrary feel', 113888), (u'contemptible reward of', 113179), (u'sinister vegetation', 459004), (u'dangled from the', 122901), (u'spiders', 472548), (u'fissure in', 178433)]


## 2 Transforming the labels

Some algorithms require that the labels (responses, target values) are also encoded as numbers. But even if a specific algorithm does not, it is good practice to represent classes with integers, as we have seen in a number of `scikit-learn` toy datasets.

In [42]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

y_encoded = label_encoder.fit_transform(y)
print y_encoded[:10]

print label_encoder.classes_

[0 1 0 2 1 2 0 0 0 2]
['EAP' 'HPL' 'MWS']


In case the target values need to be binary labels instead of integers (for multi-class problems, for instance), use `LabelBinarizer`:

In [43]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()

y_binarized = label_binarizer.fit_transform(y)
print y_binarized[:10]

print label_binarizer.classes_

[[1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 0 1]]
['EAP' 'HPL' 'MWS']


## 3 Outlook

Feature extraction has many interesting facets beyond what is discussed here, a few things for you to research or think about:

- **sparsity vs. density:** most of the vectors we have created in this notebook are _sparse_ vectors. Sparsity means that most dimensions of a vector has a `0`. At the same time, the number of dimensions grows with the vocabulary, which can easily be a six-digit number. There are ways of compressing sparse or overly long vectors into other vectors that have fewer dimensions. Those vectors are called **dense representations**.
- **Length of history for sequences:** an n-gram vectorizer does not completely discard the order of sequence items, but the so-called "history" is limited to the n-gram size. In the context of neural networks, there are methods that guarantee **infinite history** and that can truly take sequences as features.
- **integrated feature extraction:** in recent years, our community has made feature extraction part of the overall problem. This is very neat, since feature extraction can then be optimized like everything else. For instance, in a typical neural network for natural language processing, the first layers of the network actually implicitly perform feature extraction. This results in dense feature representations, or **embeddings**.